In [2]:
import numpy as np

In [3]:
from dataset_tools import image_to_patches,patches_to_image
def test_patches_to_image_and_image_to_patches(seed=None):
    if not(seed is None):
        np.random.seed(seed)
    patch_w = np.random.randint(1,40)
    patch_h = np.random.randint(1,40)
    rows = np.random.randint(1,10)
    cols = np.random.randint(1,10)
    
    img = np.random.rand(rows*patch_h,cols*patch_w,3)
    patches = image_to_patches(img,patch_w,patch_h)
    assert (patches_to_image(patches,cols,rows)-img).sum() < 1e-8,'image_to_patches->patches_to_image failed for 3 channel'
    
    patches = np.random.rand(rows*cols,patch_h,patch_w,3)
    img = patches_to_image(patches,cols,rows)
    assert (image_to_patches(img,patch_w,patch_h)-patches).sum()<1e-8,'patches_to_image->image_to_patches failed for 3 channel'
    
    patches = np.random.rand(rows*cols,patch_h,patch_w)
    img = patches_to_image(patches,cols,rows)
    assert (image_to_patches(img,patch_w,patch_h)-patches).sum()<1e-8,'patches_to_image->image_to_patches failed for 1 channel'
    
    patches = np.clip(np.random.rand(rows*cols,patch_w,patch_h,3)+100,1,10)
    img = patches_to_image(patches)
    _patches = image_to_patches(img,patch_h,patch_w)
    assert (_patches[_patches.sum((1,2,3)) > 0.01]-patches).sum()<1e-8,'patches_to_image->image_to_patches failed for any cols,rows'
    print ('test_patches_to_image_and_image_to_patches ok')
    
test_patches_to_image_and_image_to_patches(1)

test_patches_to_image_and_image_to_patches ok


In [3]:
from utils import Logger
from test_tools import *

class TestCopyLabelAlgorithm(BGAlgorithm):
    def __init__(self):
        BGAlgorithm.__init__(self)
    
    def predict_batched(self,data,labels):
        res = binarise_label(labels)
        return res
    
class TestPr1Algorithm(BGAlgorithm):
    def __init__(self):
        BGAlgorithm.__init__(self)
    
    def predict_batched(self,data,labels):
        res = binarise_label(labels).flatten()
        res[res > 0.99] = np.random.rand(res[res > 0.99].size)
        return res.reshape(labels.shape)
    

    
class Test0ApsAlgorithm(BGAlgorithm):
    def __init__(self):
        BGAlgorithm.__init__(self)
        self.balanse = []
    
    def predict_batched(self,data,labels):
        res = binarise_label(labels)
        self.balanse.append(float(res[res >0.99].size)/float(res[res<0.01].size))
        return 1-res.astype(np.int)

res = make_test(TestCopyLabelAlgorithm(),None,'data/test',0,100,(100,100),logger=Logger())
assert res['total']['aps'] > 0.999
assert res['total']['acc'] > 0.999
assert res['total']['f1'] > 0.999
res = make_test(TestPr1Algorithm(),None,'data/test',0,100,(100,100),logger=Logger())
assert res['total']['aps'] > 0.999
test0ApsAlgorithm = Test0ApsAlgorithm()
res = make_test(test0ApsAlgorithm,None,'data/test',0,100,(100,100),logger=Logger())
assert res['total']['acc'] < 1e-10
assert res['total']['f1'] < 1e-10
print res['total']['aps'] ,np.array(test0ApsAlgorithm.balanse).mean()/2.
assert abs(res['total']['aps'] - np.array(test0ApsAlgorithm.balanse).mean()/2.) < 0.001
print 'make_test ok'

/usr/local/lib/python2.7/dist-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None


0.00865918875287 0.00905305427384
make_test ok


Mapped name None to device cuda: GeForce GT 740M (0000:01:00.0)


In [6]:
from test_tools import GMMAlgorithm
from sklearn.metrics import average_precision_score
from multiprocessing import Pool
import theano.tensor as T

def mix_labels(a,b,w=0.1):
    fa = a.reshape((-1))
    fb = b.reshape((-1))
    res = fa.copy()
    inx = np.random.choice(np.arange(len(res)),int(w*len(res)),False).astype(np.int32)
    res[inx] = fb[inx]
    return res.reshape(a.shape)

def generate_test_data_for_gmmalgorithm(means,disp,lm,ld,im_size=(60,60),length=300,lw=0.1):
    ndim = len(means[0])
    def make_mixture(a,b,w=0.1):
        fa = a.reshape((-1,a.shape[-1]))
        fb = b.reshape((-1,b.shape[-1]))
        res = fa.copy()
        inx = np.random.choice(np.arange(len(res)),int(w*len(res)),False).astype(np.int32)
        res[inx] = fb[inx]
        mask = np.zeros(len(res))
        mask[inx] = 1.
        return res.reshape(a.shape),mask.reshape(a.shape[:-1])
    background = None
    for m,d in zip(means,disp):
        m = np.array(m)
        tmp = np.random.randn(length,im_size[1],im_size[0],ndim)*d+m[None,None,None,:]
        if(background is None):
            background = tmp
        else:
            background,_ = make_mixture(background,tmp)
    lm = np.array(lm)
    label_vals = np.random.randn(length,im_size[1],im_size[0],ndim)*ld+lm[None,None,None,:]
    data,labels = make_mixture(background,label_vals,lw)
    labels = mix_labels(labels,np.zeros_like(labels)+0.5)*255
    return data,labels

In [7]:
pool = Pool(4)

In [8]:
algo = GMMAlgorithm(lambda data,input_shape : T.transpose(data,(0,2,3,1)),3,pool=pool)
data,labels = generate_test_data_for_gmmalgorithm([[0.5,0.0],[0.1,0.0],[0.2,0.0]],[0.001,0.001,0.001],[-0.5,0.0],0.001,(40,40))
algo.train_batched(data[:100],labels[:100])
pred = algo.predict_batched(data[100:],labels[100:])
res = calc_prediction_metrics(pred,labels[100:],dict({'aps':average_precision_score}))
print res['aps']
assert res['aps'] > 0.98

1.0


In [9]:
algo = GMMAlgorithm(lambda data,input_shape : T.transpose(data,(0,2,3,1)),1,pool=pool)
data = np.random.randn(300,60,60,1)
labels = mix_labels(np.zeros(data.shape[:-1]),np.ones(data.shape[:-1])*255,0.5)
algo.train_batched(data[:100],labels[:100])
pred = algo.predict_batched(data[100:],labels[100:])
res = calc_prediction_metrics(pred,labels[100:],dict({'aps':average_precision_score}))
print res['aps']
assert np.abs(res['aps'] - 0.5) < 0.01

0.507993516778
